In [6]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1]))) / scale_factor

def score_submission(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    total_score = Decimal('0')
    per_n_scores = {}
    
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg'])
                     for _, row in group_data.iterrows()]
        side = get_tree_list_side_length(tree_list)
        score = side ** 2 / Decimal(str(n))
        total_score += score
        per_n_scores[n] = float(score)
    
    return float(total_score), per_n_scores

print('Scoring pre-optimized solutions...')

Scoring pre-optimized solutions...


In [7]:
# Score the bucket-of-chump solution
boc_score, boc_per_n = score_submission('preoptimized/submission.csv')
print(f'bucket-of-chump score: {boc_score:.6f}')
print(f'Target score: 68.919154')
print(f'Gap: {boc_score - 68.919154:.6f}')

bucket-of-chump score: 70.647327
Target score: 68.919154
Gap: 1.728173


In [8]:
# Score the saspav solution
saspav_score, saspav_per_n = score_submission('saspav/santa-2025.csv')
print(f'saspav score: {saspav_score:.6f}')
print(f'Target score: 68.919154')
print(f'Gap: {saspav_score - 68.919154:.6f}')

saspav score: 70.658891
Target score: 68.919154
Gap: 1.739737


In [9]:
# Find which N values have the highest scores (most room for improvement)
import matplotlib.pyplot as plt

# Use the better solution (bucket-of-chump)
per_n = boc_per_n

# Sort by score contribution
sorted_scores = sorted(per_n.items(), key=lambda x: x[1], reverse=True)
print('Top 20 N values with highest score contribution:')
for n, score in sorted_scores[:20]:
    print(f'  N={n:3d}: {score:.6f}')

print(f'\nTotal from top 20: {sum(s for _, s in sorted_scores[:20]):.6f}')
print(f'Total from rest: {sum(s for _, s in sorted_scores[20:]):.6f}')

Top 20 N values with highest score contribution:
  N=  1: 0.661250
  N=  2: 0.450779
  N=  3: 0.434745
  N=  5: 0.416850
  N=  4: 0.416545
  N=  7: 0.399897
  N=  6: 0.399610
  N=  9: 0.387415
  N=  8: 0.385407
  N= 15: 0.379203
  N= 10: 0.376630
  N= 21: 0.376451
  N= 20: 0.376057
  N= 22: 0.375258
  N= 11: 0.374924
  N= 16: 0.374128
  N= 26: 0.373997
  N= 12: 0.372724
  N= 13: 0.372294
  N= 25: 0.372144

Total from top 20: 8.076308
Total from rest: 62.571019


In [10]:
# Analyze score distribution by N ranges
small_n = sum(per_n[n] for n in range(1, 51))  # N=1-50
medium_n = sum(per_n[n] for n in range(51, 101))  # N=51-100
large_n = sum(per_n[n] for n in range(101, 151))  # N=101-150
vlarge_n = sum(per_n[n] for n in range(151, 201))  # N=151-200

print(f'Score by N range:')
print(f'  N=1-50:    {small_n:.6f}')
print(f'  N=51-100:  {medium_n:.6f}')
print(f'  N=101-150: {large_n:.6f}')
print(f'  N=151-200: {vlarge_n:.6f}')
print(f'  Total:     {small_n + medium_n + large_n + vlarge_n:.6f}')

Score by N range:
  N=1-50:    19.039870
  N=51-100:  17.627892
  N=101-150: 17.136578
  N=151-200: 16.842987
  Total:     70.647327
